In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('max_colwidth', None)

plt.style.use('fivethirtyeight')
sns.set()
sns.set_context("talk")
%matplotlib inline
pd.set_option('display.max_columns', None)

In [3]:
data_string=r"C:\Users\am513\PS-88-21-DEV\lab\aaronfolder\NHIS2009_clean.dta" 
df_og=pd.read_stata(data_string, convert_missing=True)
df=pd.read_stata(data_string)
df_og;
#df.to_csv("NHIS2009_clean.csv")

In [4]:
df_og["hi_hsb1"]=pd.to_numeric(df_og["hi_hsb1"],errors="coerce")
df_og["hi"]=pd.to_numeric(df_og["hi"],errors="coerce")
df_og["age"]=pd.to_numeric(df_og["age"],errors="coerce")
#df_og.replace
#df_og["educ"]=pd.to_numeric(df_og["age"],errors="coerce")#why did this take so long :(

In [5]:
df_og2=df_og.query("marradult == 1 & perweight != 0")
df_og2

,year,inc1,inc2,inc3,inc4,inc5,inc6,inc7,inc8,serial,hhweight,pernum,perweight,sampweight,age,marstat,sex,famsize,relate,racenew,educ,educrec1,empstat,incfam07on,health,uninsured,age2,fml,nwhite,hi,yedu,empl,hlth,inc,incmp,brooks,marradult,marradult_empl,adltempl,hi_hsb1
1,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,3,7871,1,8938.0,22029.0,29.0,Married,Female,4 people,Householder,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$0 - $34,999",Very Good,Not covered,841.0,1.0,0.0,0.0,14,0.0,4.0,19282.931641,17500.0,1.0,1.0,0.0,1.0,NaN
4,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,3,7871,4,8967.0,0.0,35.0,Married,Male,4 people,Spouse,White,Grade 11,Grade 11,Working for pay at job/business,"$0 - $34,999",Very Good,Not covered,1225.0,0.0,0.0,0.0,11,1.0,4.0,19282.931641,17500.0,1.0,1.0,1.0,1.0,0.0
7,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,5,7871,1,8905.0,0.0,32.0,Married,Male,4 people,Householder,White,High school graduate,Grade 12,Working for pay at job/business,"$100,000 and over",Good,Covered,1024.0,0.0,0.0,1.0,12,1.0,3.0,167844.531250,150000.0,1.0,1.0,1.0,2.0,1.0
8,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,5,7871,2,8889.0,22190.0,34.0,Married,Female,4 people,Spouse,White,"Bachelor's degree (BA, AB, BS, BBA)",4 years of college/Bachelor's degree,Working for pay at job/business,"$100,000 and over",Good,Covered,1156.0,1.0,0.0,1.0,16,1.0,3.0,167844.531250,150000.0,1.0,1.0,1.0,2.0,NaN
11,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,6,7871,1,8378.0,19284.0,65.0,Married,Male,2 people,Householder,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$35,000 - $49,999",Good,Covered,4225.0,0.0,0.0,1.0,14,0.0,3.0,41679.343750,42500.0,0.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80620,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41173,2220,2,2496.0,5732.0,57.0,Married,Female,2 people,Spouse,White,High school graduate,Grade 12,Not in labor force,"$100,000 and over",Very Good,Covered,3249.0,1.0,0.0,1.0,12,0.0,4.0,167844.531250,150000.0,1.0,1.0,0.0,1.0,NaN
80624,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41175,2624,1,3135.0,7200.0,67.0,Married,Female,2 people,Householder,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$50,000 - $74,999",Good,Covered,4489.0,1.0,0.0,1.0,14,0.0,3.0,61102.972656,62500.0,0.0,1.0,0.0,0.0,NaN
80625,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41175,2624,2,3022.0,0.0,68.0,Married,Male,2 people,Spouse,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$50,000 - $74,999",Fair,Covered,4624.0,0.0,0.0,1.0,14,0.0,2.0,61102.972656,62500.0,0.0,1.0,0.0,0.0,1.0
80626,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41176,2200,1,2532.0,18062.0,62.0,Married,Male,7 people,Spouse,White,Grade 9,Grade 9,Working for pay at job/business,"$100,000 and over",Poor,Not covered,3844.0,0.0,0.0,0.0,9,1.0,1.0,167844.531250,150000.0,0.0,1.0,1.0,2.0,0.0


In [5]:
df2=df.query("marradult == 1 & perweight != 0") #keep if marradult==1 & perweight!=0 
df2;

In [7]:
df_og3=df_og2.groupby("serial").sum().reset_index()[["serial","hi_hsb1"]]

In [8]:
df_og4=df_og2.merge(df_og3,how="left",on="serial")
df_og4

,year,inc1,inc2,inc3,inc4,inc5,inc6,inc7,inc8,serial,hhweight,pernum,perweight,sampweight,age,marstat,sex,famsize,relate,racenew,educ,educrec1,empstat,incfam07on,health,uninsured,age2,fml,nwhite,hi,yedu,empl,hlth,inc,incmp,brooks,marradult,marradult_empl,adltempl,hi_hsb1_x,hi_hsb1_y
0,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,3,7871,1,8938.0,22029.0,29.0,Married,Female,4 people,Householder,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$0 - $34,999",Very Good,Not covered,841.0,1.0,0.0,0.0,14,0.0,4.0,19282.931641,17500.0,1.0,1.0,0.0,1.0,NaN,0.0
1,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,3,7871,4,8967.0,0.0,35.0,Married,Male,4 people,Spouse,White,Grade 11,Grade 11,Working for pay at job/business,"$0 - $34,999",Very Good,Not covered,1225.0,0.0,0.0,0.0,11,1.0,4.0,19282.931641,17500.0,1.0,1.0,1.0,1.0,0.0,0.0
2,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,5,7871,1,8905.0,0.0,32.0,Married,Male,4 people,Householder,White,High school graduate,Grade 12,Working for pay at job/business,"$100,000 and over",Good,Covered,1024.0,0.0,0.0,1.0,12,1.0,3.0,167844.531250,150000.0,1.0,1.0,1.0,2.0,1.0,1.0
3,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,5,7871,2,8889.0,22190.0,34.0,Married,Female,4 people,Spouse,White,"Bachelor's degree (BA, AB, BS, BBA)",4 years of college/Bachelor's degree,Working for pay at job/business,"$100,000 and over",Good,Covered,1156.0,1.0,0.0,1.0,16,1.0,3.0,167844.531250,150000.0,1.0,1.0,1.0,2.0,NaN,1.0
4,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,6,7871,1,8378.0,19284.0,65.0,Married,Male,2 people,Householder,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$35,000 - $49,999",Good,Covered,4225.0,0.0,0.0,1.0,14,0.0,3.0,41679.343750,42500.0,0.0,1.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29967,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41173,2220,2,2496.0,5732.0,57.0,Married,Female,2 people,Spouse,White,High school graduate,Grade 12,Not in labor force,"$100,000 and over",Very Good,Covered,3249.0,1.0,0.0,1.0,12,0.0,4.0,167844.531250,150000.0,1.0,1.0,0.0,1.0,NaN,1.0
29968,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41175,2624,1,3135.0,7200.0,67.0,Married,Female,2 people,Householder,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$50,000 - $74,999",Good,Covered,4489.0,1.0,0.0,1.0,14,0.0,3.0,61102.972656,62500.0,0.0,1.0,0.0,0.0,NaN,1.0
29969,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41175,2624,2,3022.0,0.0,68.0,Married,Male,2 people,Spouse,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$50,000 - $74,999",Fair,Covered,4624.0,0.0,0.0,1.0,14,0.0,2.0,61102.972656,62500.0,0.0,1.0,0.0,0.0,1.0,1.0
29970,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41176,2200,1,2532.0,18062.0,62.0,Married,Male,7 people,Spouse,White,Grade 9,Grade 9,Working for pay at job/business,"$100,000 and over",Poor,Not covered,3844.0,0.0,0.0,0.0,9,1.0,1.0,167844.531250,150000.0,0.0,1.0,1.0,2.0,0.0,0.0


In [10]:
df_og5=df_og4.drop("hi_hsb1_x",axis=1).rename(columns={"hi_hsb1_y":"hi_hsb"})

In [11]:
df_og5_5=df_og5[["serial","hi_hsb"]].set_index("serial")
df_og5_5;#.dtypes #by serial: egen hi_hsb = mean(hi_hsb1)

In [12]:
df_og6=df_og5.dropna(subset=["hi_hsb","hi"],how="any",axis="rows")
df_og6

,year,inc1,inc2,inc3,inc4,inc5,inc6,inc7,inc8,serial,hhweight,pernum,perweight,sampweight,age,marstat,sex,famsize,relate,racenew,educ,educrec1,empstat,incfam07on,health,uninsured,age2,fml,nwhite,hi,yedu,empl,hlth,inc,incmp,brooks,marradult,marradult_empl,adltempl,hi_hsb
0,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,3,7871,1,8938.0,22029.0,29.0,Married,Female,4 people,Householder,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$0 - $34,999",Very Good,Not covered,841.0,1.0,0.0,0.0,14,0.0,4.0,19282.931641,17500.0,1.0,1.0,0.0,1.0,0.0
1,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,3,7871,4,8967.0,0.0,35.0,Married,Male,4 people,Spouse,White,Grade 11,Grade 11,Working for pay at job/business,"$0 - $34,999",Very Good,Not covered,1225.0,0.0,0.0,0.0,11,1.0,4.0,19282.931641,17500.0,1.0,1.0,1.0,1.0,0.0
2,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,5,7871,1,8905.0,0.0,32.0,Married,Male,4 people,Householder,White,High school graduate,Grade 12,Working for pay at job/business,"$100,000 and over",Good,Covered,1024.0,0.0,0.0,1.0,12,1.0,3.0,167844.531250,150000.0,1.0,1.0,1.0,2.0,1.0
3,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,5,7871,2,8889.0,22190.0,34.0,Married,Female,4 people,Spouse,White,"Bachelor's degree (BA, AB, BS, BBA)",4 years of college/Bachelor's degree,Working for pay at job/business,"$100,000 and over",Good,Covered,1156.0,1.0,0.0,1.0,16,1.0,3.0,167844.531250,150000.0,1.0,1.0,1.0,2.0,1.0
4,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,6,7871,1,8378.0,19284.0,65.0,Married,Male,2 people,Householder,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$35,000 - $49,999",Good,Covered,4225.0,0.0,0.0,1.0,14,0.0,3.0,41679.343750,42500.0,0.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29967,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41173,2220,2,2496.0,5732.0,57.0,Married,Female,2 people,Spouse,White,High school graduate,Grade 12,Not in labor force,"$100,000 and over",Very Good,Covered,3249.0,1.0,0.0,1.0,12,0.0,4.0,167844.531250,150000.0,1.0,1.0,0.0,1.0,1.0
29968,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41175,2624,1,3135.0,7200.0,67.0,Married,Female,2 people,Householder,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$50,000 - $74,999",Good,Covered,4489.0,1.0,0.0,1.0,14,0.0,3.0,61102.972656,62500.0,0.0,1.0,0.0,0.0,1.0
29969,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41175,2624,2,3022.0,0.0,68.0,Married,Male,2 people,Spouse,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$50,000 - $74,999",Fair,Covered,4624.0,0.0,0.0,1.0,14,0.0,2.0,61102.972656,62500.0,0.0,1.0,0.0,0.0,1.0
29970,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41176,2200,1,2532.0,18062.0,62.0,Married,Male,7 people,Spouse,White,Grade 9,Grade 9,Working for pay at job/business,"$100,000 and over",Poor,Not covered,3844.0,0.0,0.0,0.0,9,1.0,1.0,167844.531250,150000.0,0.0,1.0,1.0,2.0,0.0


In [13]:
to_drop=[
17292,
40202,
30706,
4326,
12054,
22116,
6254,
3487,
11669,
28051,
35481,
8153,
9464,
6659,
6713,
23977,
289,
297,
2748,
39073,
15857,
13708,
6580,
18688,
2610,
11302,
15470,
25851,
15368,
22092,
28266,
30999,
20588,
21166,
10611,
8317,
35601,
5859,
17836,
8479,
24118,
5858,
29981,
9174,
36905,
21169,
9358,
19397,
28835,
29784,
24591,
8137,
4396,
26444,
3978,
2463,
24533,
847,
34905,
26602,
18098,
19080,
13759,
4931,
13332,
11538,
15599,
25225,
22636,
18259,
10329,
22341,
27758,
11906,
16966,
11950,
20720,
22743,
17672,
40794,
1891,
4160,
32355,
11538,
5462,
40801,
34589,
25094,
15412,
6689,
9268,
19300,
29482,
27331,
7181,
5005,
31265,
5938,
13708,
13728,
13186,
33220,
38457,
13936,
12479,
22525,
31406,
24890,
26403,
34368,
2262,
14275,
26437,
1497,
30913,
13777,
32412,
14540,
30547,
21201,
12030,
4518,
14275,
2409,
6766,
4790,
32590,
11119,
33686,
34894,
29668,
3136,
3725,
40957,
3330,
33150,
4875,
29392,
23251,
7743,
819,
27927,
2610,
15775,
32756,
7969,
17881,
17912,
34099,
17137,
7969,
10296,
13673,
8431,
21462,
27080,
7020,
32355,
26237,
8205,
25727,
26826,
20842,
13316,
29921,
2960,
2748,
34748,
29534,
31658,
6254,
22564,
2111,
7539,
9510,
17329,
34229,
8907,
7645,
8910,
7346,
10061,
19712,
24606,
22331,
16639,
21164,
36081,
21421,
29884,
20349,
12603,
21622,
40529,
31008,
14397,
23956,
32457,
32256,
23775,
19439,
2241,
38695,
22743,
30273,
29104,
6233
]

In [31]:
df_og6=df_og6[~df_og6["serial"].isin(to_drop)]
df_og6

,year,inc1,inc2,inc3,inc4,inc5,inc6,inc7,inc8,serial,hhweight,pernum,perweight,sampweight,age,marstat,sex,famsize,relate,racenew,educ,educrec1,empstat,incfam07on,health,uninsured,age2,fml,nwhite,hi,yedu,empl,hlth,inc,incmp,brooks,marradult,marradult_empl,adltempl,hi_hsb,female
0,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,3,7871,1,8938.0,22029.0,29.0,Married,Female,4 people,Householder,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$0 - $34,999",Very Good,Not covered,841.0,1.0,0.0,0.0,14,0.0,4.0,19282.931641,17500.0,1.0,1.0,0.0,1.0,0.0,1
1,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,3,7871,4,8967.0,0.0,35.0,Married,Male,4 people,Spouse,White,Grade 11,Grade 11,Working for pay at job/business,"$0 - $34,999",Very Good,Not covered,1225.0,0.0,0.0,0.0,11,1.0,4.0,19282.931641,17500.0,1.0,1.0,1.0,1.0,0.0,1
2,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,5,7871,1,8905.0,0.0,32.0,Married,Male,4 people,Householder,White,High school graduate,Grade 12,Working for pay at job/business,"$100,000 and over",Good,Covered,1024.0,0.0,0.0,1.0,12,1.0,3.0,167844.531250,150000.0,1.0,1.0,1.0,2.0,1.0,1
3,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,5,7871,2,8889.0,22190.0,34.0,Married,Female,4 people,Spouse,White,"Bachelor's degree (BA, AB, BS, BBA)",4 years of college/Bachelor's degree,Working for pay at job/business,"$100,000 and over",Good,Covered,1156.0,1.0,0.0,1.0,16,1.0,3.0,167844.531250,150000.0,1.0,1.0,1.0,2.0,1.0,1
4,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,6,7871,1,8378.0,19284.0,65.0,Married,Male,2 people,Householder,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$35,000 - $49,999",Good,Covered,4225.0,0.0,0.0,1.0,14,0.0,3.0,41679.343750,42500.0,0.0,1.0,0.0,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29967,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41173,2220,2,2496.0,5732.0,57.0,Married,Female,2 people,Spouse,White,High school graduate,Grade 12,Not in labor force,"$100,000 and over",Very Good,Covered,3249.0,1.0,0.0,1.0,12,0.0,4.0,167844.531250,150000.0,1.0,1.0,0.0,1.0,1.0,1
29968,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41175,2624,1,3135.0,7200.0,67.0,Married,Female,2 people,Householder,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$50,000 - $74,999",Good,Covered,4489.0,1.0,0.0,1.0,14,0.0,3.0,61102.972656,62500.0,0.0,1.0,0.0,0.0,1.0,1
29969,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41175,2624,2,3022.0,0.0,68.0,Married,Male,2 people,Spouse,White,"Some college, no degree",1 to 3 years of college,Not in labor force,"$50,000 - $74,999",Fair,Covered,4624.0,0.0,0.0,1.0,14,0.0,2.0,61102.972656,62500.0,0.0,1.0,0.0,0.0,1.0,1
29970,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41176,2200,1,2532.0,18062.0,62.0,Married,Male,7 people,Spouse,White,Grade 9,Grade 9,Working for pay at job/business,"$100,000 and over",Poor,Not covered,3844.0,0.0,0.0,0.0,9,1.0,1.0,167844.531250,150000.0,0.0,1.0,1.0,2.0,0.0,1


In [19]:
males=[
27076,
32501,
36537,
12722,
5958,
18162,
35909,
34396,
17814,
10369,
37415,
36924,
20036,
19066,
22639,
34048,
35904,
29606,
24936,
34048,
21939,
29126,
29542,
2605,
15837,
35956,
22639,
2605,
21666,
2265,
15837,
]

In [20]:
df_og6["female"]=1

C:\Users\am513\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
df_og6.loc[df_og6["serial"].isin(males),"female"]=0 #by serial: egen female = total(fml)

C:\Users\am513\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [22]:
df_og6["female"].value_counts()

1    29734
0       31
Name: female, dtype: int64

In [23]:
df_og7=df_og6.query("female == 1").drop("female",axis="columns") #keep if female==1     drop female
df_og7;

In [24]:
df_og7.dtypes

year               float32
inc1               float32
inc2               float32
inc3               float32
inc4               float32
inc5               float32
inc6               float32
inc7               float32
inc8               float32
serial               int32
hhweight             int32
pernum                int8
perweight          float64
sampweight         float64
age                float64
marstat           category
sex               category
famsize           category
relate            category
racenew           category
educ              category
educrec1          category
empstat           category
incfam07on        category
health            category
uninsured         category
age2               float32
fml                float32
nwhite             float32
hi                 float32
yedu                object
empl               float32
hlth               float32
inc                float32
incmp              float32
brooks             float32
marradult          float32
m

In [25]:
df_og8=df_og7.query("age>=26 & age<=59 & marradult==1 & adltempl>=1")

In [26]:
df_og9=df_og8.groupby(["serial"]).filter(lambda x: len(x)>1) #by serial: gen n = _N   keep if n>1 (This is also really bad code)

In [27]:
husband_somehi_ss=len(df_og9.query("hi == 1 & fml == 0"));
husband_nohi_ss=len(df_og9.query("hi == 0 & fml == 0"));
husband_somehi_age=np.round(np.mean(df_og9.query("hi == 1 & fml == 0")["age"]),decimals=2);
husband_nohi_age=np.round(np.mean(df_og9.query("hi == 0 & fml == 0")["age"]),decimals=2);
husband_diff_age=df_og9.query("hi == 0 & fml == 0")["age"].sem()-(df_og9.query("hi == 1 & fml == 0")["age"]).sem()
husband_somehi_edu=np.round(np.mean(df_og9.query("hi == 1 & fml == 0")["yedu"]),decimals=2)
husband_nohi_edu=np.round(np.mean(df_og9.query("hi == 0 & fml == 0")["yedu"]),decimals=2)
husband_diff_edu=np.round(df_og9.query("hi == 0 & fml == 0")["yedu"].sem()-(df_og9.query("hi == 1 & fml == 0")["yedu"]).sem(),decimals=2)
husband_diff_edu_diff=np.round(husband_somehi_edu-husband_nohi_edu,decimals=2)
df_og9["famsize num only"]=df_og9["famsize"].str.split(expand=True)[0]
df_og9["famsize num only"]=pd.to_numeric(df_og9["famsize num only"],errors="coerce")
husband_nohi_famsize=np.round(np.array(df_og9.query("hi == 0 & fml == 0")["famsize num only"],dtype=np.float).mean(),decimals=2)
husband_somehi_famsize=np.round(np.array(df_og9.query("hi == 1 & fml == 0")["famsize num only"],dtype=np.float).mean(),decimals=2)
husband_diff_famsize=np.round(husband_somehi_famsize-husband_nohi_famsize,decimals=2)
husband_diff_famsize_diff=np.round(df_og9.query("hi == 0 & fml == 0")["famsize num only"].sem()-(df_og9.query("hi == 1 & fml == 0")["famsize num only"]).sem(),decimals=2)
husband_nohi_emp=np.round(np.array(df_og9.query("hi == 0 & fml == 0")["empl"],dtype=np.float).mean(),decimals=2)
husband_somehi_emp=np.round(np.array(df_og9.query("hi == 1 & fml == 0")["empl"],dtype=np.float).mean(),decimals=2)
husband_diff_emp_diff=np.round(husband_somehi_emp-husband_nohi_emp,decimals=2)
husband_diff_edu_sem=np.round(df_og9.query("hi == 0 & fml == 0")["empl"].sem()-(df_og9.query("hi == 1 & fml == 0")["empl"]).sem(),decimals=2)
husband_nohi_inc=np.round(np.array(df_og9.query("hi == 0 & fml == 0")["inc"],dtype=np.float).mean(),decimals=2)
husband_somehi_inc=np.round(np.array(df_og9.query("hi == 1 & fml == 0")["inc"],dtype=np.float).mean(),decimals=2)
husband_diff_inc_diff=np.round(husband_somehi_inc-husband_nohi_inc,decimals=2)
husband_diff_inc_sem=np.round(df_og9.query("hi == 0 & fml == 0")["inc"].sem()-(df_og9.query("hi == 1 & fml == 0")["inc"]).sem(),decimals=2)
husband_nohi_nwhite=np.round(np.array(df_og9.query("hi == 0 & fml == 0")["nwhite"],dtype=np.float).mean(),decimals=2)
husband_somehi_nwhite=np.round(np.array(df_og9.query("hi == 1 & fml == 0")["nwhite"],dtype=np.float).mean(),decimals=2)
husband_diff_nwhite_diff=np.round(husband_somehi_nwhite-husband_nohi_nwhite,decimals=2)
husband_diff_nwhite_sem=np.round(df_og9.query("hi == 0 & fml == 0")["nwhite"].sem()-(df_og9.query("hi == 1 & fml == 0")["nwhite"]).sem(),decimals=2)
husband_nohi_hlth=np.round(np.array(df_og9.query("hi == 0 & fml == 0")["hlth"],dtype=np.float).mean(),decimals=2)
husband_somehi_hlth=np.round(np.array(df_og9.query("hi == 1 & fml == 0")["hlth"],dtype=np.float).mean(),decimals=2)
husband_nohi_hlth_sd=np.round(np.std(np.array(df_og9.query("hi == 0 & fml == 0")["hlth"])),decimals=2)
husband_somehi_hlth_sd=np.round(np.std(np.array(df_og9.query("hi == 1 & fml == 0")["hlth"])),decimals=2)
husband_diff_hlth_diff=np.round(husband_somehi_hlth-husband_nohi_hlth,decimals=2)
husband_diff_hlth_sem=np.round(df_og9.query("hi == 0 & fml == 0")["hlth"].sem()-(df_og9.query("hi == 1 & fml == 0")["hlth"]).sem(),decimals=2)

In [28]:
print("Husbands")
pd.DataFrame(data={' ': ["Health index", "Nonwhite","Age","Education","Family Size","Employed","Family Income","Sample Size"],
                   'Some HI (1)': [f"{husband_somehi_hlth} [{np.round(husband_somehi_hlth_sd)}]", husband_somehi_nwhite,husband_somehi_age,husband_somehi_edu,husband_somehi_famsize,husband_somehi_emp,husband_somehi_inc,husband_somehi_ss],
                  'No HI (0)': [f"{husband_nohi_hlth} [{np.round(husband_nohi_hlth_sd)}]", husband_nohi_nwhite,husband_nohi_age,husband_nohi_edu,husband_nohi_famsize,husband_nohi_emp,husband_nohi_inc,husband_nohi_ss],
                  'Difference (3)': ["{0} ({1})".format(husband_diff_hlth_diff,husband_diff_hlth_sem), "{0} ({1})".format(husband_diff_nwhite_diff,husband_diff_nwhite_sem),"{0} ({1})".format(np.round(husband_somehi_age-husband_nohi_age,decimals=2),np.round(husband_diff_age,decimals=2)),"{0} ({1})".format(husband_diff_edu_diff,husband_diff_edu),"{0} ({1})".format(husband_diff_famsize,husband_diff_famsize_diff),"{0} ({1})".format(husband_diff_emp_diff,husband_diff_edu_sem),"{0} ({1})".format(husband_diff_inc_diff,husband_diff_inc_sem)," "]})

Husbands


,,Some HI (1),No HI (0),Difference (3)
0,Health index,3.98 [1.0],3.7 [1.0],0.28 (0.02)
1,Nonwhite,0.2,0.19,0.01 (0.01)
2,Age,44.16,41.27,2.89 (0.12)
3,Education,14.13,11.21,2.92 (0.06)
4,Family Size,3.55,4.06,-0.51 (0.02)
5,Employed,0.92,0.85,0.07 (0.01)
6,Family Income,104002,43636,60366.42 (294.68)
7,Sample Size,7866,1529,


In [29]:
wife_somehi_ss=len(df_og9.query("hi == 1 & fml == 1"));
wife_nohi_ss=len(df_og9.query("hi == 0 & fml == 1"));
wife_somehi_age=np.round(np.mean(df_og9.query("hi == 1 & fml == 1")["age"]),decimals=2);
wife_nohi_age=np.round(np.mean(df_og9.query("hi == 0 & fml == 1")["age"]),decimals=2);
wife_diff_age=df_og9.query("hi == 0 & fml == 1")["age"].sem()-(df_og9.query("hi == 1 & fml == 1")["age"]).sem()
wife_somehi_edu=np.round(np.mean(df_og9.query("hi == 1 & fml == 1")["yedu"]),decimals=2)
wife_nohi_edu=np.round(np.mean(df_og9.query("hi == 0 & fml == 1")["yedu"]),decimals=2)
wife_diff_edu=np.round(df_og9.query("hi == 0 & fml == 1")["yedu"].sem()-(df_og9.query("hi == 1 & fml == 1")["yedu"]).sem(),decimals=2)
wife_diff_edu_diff=np.round(wife_somehi_edu-wife_nohi_edu,decimals=2)
df_og9["famsize num only"]=df_og9["famsize"].str.split(expand=True)[0]
df_og9["famsize num only"]=pd.to_numeric(df_og9["famsize num only"],errors="coerce")
wife_nohi_famsize=np.round(np.array(df_og9.query("hi == 0 & fml == 1")["famsize num only"],dtype=np.float).mean(),decimals=2)
wife_somehi_famsize=np.round(np.array(df_og9.query("hi == 1 & fml == 1")["famsize num only"],dtype=np.float).mean(),decimals=2)
wife_diff_famsize=np.round(wife_somehi_famsize-wife_nohi_famsize,decimals=2)
wife_diff_famsize_diff=np.round(df_og9.query("hi == 0 & fml == 1")["famsize num only"].sem()-(df_og9.query("hi == 1 & fml == 0")["famsize num only"]).sem(),decimals=2)
wife_nohi_emp=np.round(np.array(df_og9.query("hi == 0 & fml == 1")["empl"],dtype=np.float).mean(),decimals=2)
wife_somehi_emp=np.round(np.array(df_og9.query("hi == 1 & fml == 1")["empl"],dtype=np.float).mean(),decimals=2)
wife_diff_emp_diff=np.round(wife_somehi_emp-wife_nohi_emp,decimals=2)
wife_diff_edu_sem=np.round(df_og9.query("hi == 0 & fml == 1")["empl"].sem()-(df_og9.query("hi == 1 & fml == 0")["empl"]).sem(),decimals=2)
wife_nohi_inc=np.round(np.array(df_og9.query("hi == 0 & fml == 1")["inc"],dtype=np.float).mean(),decimals=2)
wife_somehi_inc=np.round(np.array(df_og9.query("hi == 1 & fml == 1")["inc"],dtype=np.float).mean(),decimals=2)
wife_diff_inc_diff=np.round(wife_somehi_inc-wife_nohi_inc,decimals=2)
wife_diff_inc_sem=np.round(df_og9.query("hi == 0 & fml == 1")["inc"].sem()-(df_og9.query("hi == 1 & fml == 1")["inc"]).sem(),decimals=2)
wife_nohi_nwhite=np.round(np.array(df_og9.query("hi == 0 & fml == 1")["nwhite"],dtype=np.float).mean(),decimals=2)
wife_somehi_nwhite=np.round(np.array(df_og9.query("hi == 1 & fml == 1")["nwhite"],dtype=np.float).mean(),decimals=2)
wife_diff_nwhite_diff=np.round(wife_somehi_nwhite-wife_nohi_nwhite,decimals=2)
wife_diff_nwhite_sem=np.round(df_og9.query("hi == 0 & fml == 1")["nwhite"].sem()-(df_og9.query("hi == 1 & fml == 1")["nwhite"]).sem(),decimals=2)
wife_nohi_hlth=np.round(np.array(df_og9.query("hi == 0 & fml == 1")["hlth"],dtype=np.float).mean(),decimals=2)
wife_somehi_hlth=np.round(np.array(df_og9.query("hi == 1 & fml == 1")["hlth"],dtype=np.float).mean(),decimals=2)
wife_nohi_hlth_sd=np.round(np.std(np.array(df_og9.query("hi == 0 & fml == 1")["hlth"])),decimals=2)
wife_somehi_hlth_sd=np.round(np.std(np.array(df_og9.query("hi == 1 & fml == 1")["hlth"])),decimals=2)
wife_diff_hlth_diff=np.round(wife_somehi_hlth-wife_nohi_hlth,decimals=2)
wife_diff_hlth_sem=np.round(df_og9.query("hi == 0 & fml == 1")["hlth"].sem()-(df_og9.query("hi == 1 & fml == 1")["hlth"]).sem(),decimals=2)

In [30]:
print("Wives")
wives=pd.DataFrame(data={' ': ["Health index", "Nonwhite","Age","Education","Family Size","Employed","Family Income","Sample Size"],
                   'Some HI (1)': ["{0} [{1}]".format(wife_somehi_hlth,wife_somehi_hlth_sd), wife_somehi_nwhite,wife_somehi_age,wife_somehi_edu,wife_somehi_famsize,wife_somehi_emp,wife_somehi_inc,wife_somehi_ss],
                  'No HI (0)': ["{0} [{1}]".format(wife_nohi_hlth,wife_nohi_hlth_sd), wife_nohi_nwhite,wife_nohi_age,wife_nohi_edu,wife_nohi_famsize,wife_nohi_emp,wife_nohi_inc,wife_nohi_ss],
                  'Difference (3)': ["{0} ({1})".format(wife_diff_hlth_diff,wife_diff_hlth_sem), "{0} ({1})".format(wife_diff_nwhite_diff,wife_diff_nwhite_sem),"{0} ({1})".format(np.round(wife_somehi_age-wife_nohi_age,decimals=2),np.round(wife_diff_age,decimals=2)),"{0} ({1})".format(wife_diff_edu_diff,wife_diff_edu),"{0} ({1})".format(wife_diff_famsize,wife_diff_famsize_diff),"{0} ({1})".format(wife_diff_emp_diff,wife_diff_edu_sem),"{0} ({1})".format(wife_diff_inc_diff,wife_diff_inc_sem)," "]})
display(wives)
print("Notes: This  table  reports  average  characteristics  for  insured  and  uninsured  marriedcouples in the 2009 National Health Interview Survey (NHIS). Columns (1), (2), (4), and(5) show average characteristics of the group of individuals specified by the column heading.Columns (3) and (6) report the difference between the average characteristic for individualswith and without health insurance (HI). Standard deviations are in brackets; standard errorsare reported in parentheses.")

Wives


,,Some HI (1),No HI (0),Difference (3)
0,Health index,3.99 [0.9300000071525574],3.61 [1.0199999809265137],0.38 (0.02)
1,Nonwhite,0.2,0.18,0.02 (0.01)
2,Age,42.15,39.52,2.63 (0.12)
3,Education,14.27,11.36,2.91 (0.06)
4,Family Size,3.55,4.07,-0.52 (0.03)
5,Employed,0.76,0.54,0.22 (0.01)
6,Family Income,103364,43641.4,59722.24 (307.41)
7,Sample Size,7950,1445,


Notes: This  table  reports  average  characteristics  for  insured  and  uninsured  marriedcouples in the 2009 National Health Interview Survey (NHIS). Columns (1), (2), (4), and(5) show average characteristics of the group of individuals specified by the column heading.Columns (3) and (6) report the difference between the average characteristic for individualswith and without health insurance (HI). Standard deviations are in brackets; standard errorsare reported in parentheses.
